<div style="display: flex; background-color: #7a310c;" >
<h1 style="margin: auto; padding: 30px; ">ANALYSE DES VENTES DE LA LIBRAIRIE LAPAGE</h1>
</div>

<center>
<img src="C:\\Users\yannf\OneDrive\SWSetup\Bureau\Cours\Formation_data_analyse\projets\projet_9\ressources\librairy_image_hq.jpg" width="400"/>
</center>

In [1]:
# color hexa : #7a310c / #a75d38

<div style="display: flex; background-color: #a75d38;" >
<h1 style="margin: auto; padding: 20px; ">1. Ouverture et chargement des fichiers</h1>
</div>

In [2]:
#importation des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re

In [3]:
#Ouverture des fichiers en prenant soin de parser la date dans le fichier contenant les dates de transactions
path = "C:\\Users\\yannf\\OneDrive\\SWSetup\\Bureau\\Cours\\Formation_data_analyse\\projets\\projet_9\\input\\Clean Data"
path_2 = "C:\\Users\\yannf\\OneDrive\\SWSetup\\Bureau\\Cours\\Formation_data_analyse\\projets\\projet_9\\input\\True Data\\DAN-P6-donnees"
df_customers = pd.read_csv(f"{path_2}\\customers.csv", sep=",")
df_products = pd.read_csv(f"{path_2}\\products.csv", sep=",")
df_transactions = pd.read_csv(f"{path_2}\\Transactions.csv", sep=",", parse_dates=True)

<div style="display: flex; background-color: #7a310c;" >
<h1 style="margin: auto; padding: 20px; ">2. Contenu et dimensions des fichiers</h1>
</div>

<div style="display: flex; background-color: #a75d38;" >
<h1 style="margin: auto; padding: 20px; ">2.1 - Fichiers clients</h1>
</div>

In [4]:
# Dimensions du fichiers
print(f"Le fichier à {df_customers.shape[0]} lignes et {df_customers.shape[1]} colonnes")
print("Voici le type des colonnes :\n",df_customers.dtypes)
print(df_customers.head())

Le fichier à 8623 lignes et 3 colonnes
Voici le type des colonnes :
 client_id    object
sex          object
birth         int64
dtype: object
  client_id sex  birth
0    c_4410   f   1967
1    c_7839   f   1975
2    c_1699   f   1984
3    c_5961   f   1962
4    c_5320   m   1943


<div style="display: flex; background-color: #a75d38;" >
<h1 style="margin: auto; padding: 20px; ">2.2 - Fichiers produits</h1>
</div>

In [5]:
# Dimensions du fichiers
print(f"Le fichier à {df_products.shape[0]} lignes et {df_products.shape[1]} colonnes")
print("Voici le type des colonnes :\n",df_products.dtypes)
print(df_products.head())

Le fichier à 3287 lignes et 3 colonnes
Voici le type des colonnes :
 id_prod     object
price      float64
categ        int64
dtype: object
  id_prod  price  categ
0  0_1421  19.99      0
1  0_1368   5.13      0
2   0_731  17.99      0
3   1_587   4.99      1
4  0_1507   3.99      0


<div style="display: flex; background-color: #a75d38;" >
<h1 style="margin: auto; padding: 20px; ">2.3 - Fichiers transactions</h1>
</div>

In [6]:
# Dimensions du fichiers
print(f"Le fichier à {df_transactions.shape[0]} lignes et {df_transactions.shape[1]} colonnes")
print("Voici le type des colonnes :\n",df_transactions.dtypes)
print(df_transactions.head())

Le fichier à 679532 lignes et 4 colonnes
Voici le type des colonnes :
 id_prod       object
date          object
session_id    object
client_id     object
dtype: object
  id_prod                        date session_id client_id
0  0_1518  2022-05-20 13:21:29.043970   s_211425     c_103
1   1_251  2022-02-02 07:55:19.149409   s_158752    c_8534
2  0_1277  2022-06-18 15:44:33.155329   s_225667    c_6714
3   2_209  2021-06-24 04:19:29.835891    s_52962    c_6941
4  0_1509  2023-01-11 08:22:08.194479   s_325227    c_4232


<div style="display: flex; background-color: #7a310c;" >
<h1 style="margin: auto; padding: 20px; ">3. Nettoyage et préparation des données</h1>
</div>

<div style="display: flex; background-color: #a75d38;" >
<h1 style="margin: auto; padding: 20px; ">3.1 - Fichiers clients</h1>
</div>

In [7]:
#vérification de la structure des colonnes
print(f"Nous avons {len(df_customers)-len(df_customers[df_customers['client_id'].str.startswith('c_')])} identifiant client qui sont au mauvais format.")
print(f"Il y a {len(df_customers['client_id'].unique())} identifiants uniques, soit autant que d'enregistrement de client")
print(f"Les sexes différents des clients sont {' et '.join(df_customers['sex'].unique())}.")

Nous avons 2 identifiant client qui sont au mauvais format.
Il y a 8623 identifiants uniques, soit autant que d'enregistrement de client
Les sexes différents des clients sont f et m.


In [8]:
#suppression des identifiants au mauvais format car relier a des tests qui n'ont pas d'impact sur les analyses
df_customers = df_customers.loc[~df_customers["client_id"].isin(["ct_0", "ct_1"])]

In [9]:
#correction des identifiants au mauvais format
df_customers[df_customers['client_id'].str.startswith('c_') ==False]
df_customers_test = df_customers[df_customers['client_id'].str.startswith('c_') ==True].copy()
df_customers_test["client_number"] = df_customers_test["client_id"].str[2:]
df_customers_test["client_number"]  = pd.to_numeric(df_customers_test["client_number"])
#remplace les valeurs d'identifiants clients étant au mauvais format en incrémentant à partir du dernier pour celui en doublon
df_customers.loc[df_customers['client_id'] == "ct_0", "client_id"] = "c_0"
df_customers.loc[df_customers['client_id'] == "ct_1", "client_id"] = f"c_{df_customers_test['client_number'].max()+1}"

In [10]:
# vérifie qu'il n'y a pas de valeur d'ages atypiques  (plus de 100 ans ou moins de 18 ans)
print(f"Il y a {len(df_customers.loc[(df_customers['birth'] <1925) |(df_customers['birth'] > 2007)])} clients qui ont un age atypiques")
# modifie les année de naissance en age
df_customers["birth"]= datetime.datetime.today().year - df_customers["birth"]
df_customers.rename(columns={"birth": "age"}, inplace=True)
df_customers.head(3)

Il y a 0 clients qui ont un age atypiques


,client_id,sex,age
0,c_4410,f,58
1,c_7839,f,50
2,c_1699,f,41


In [11]:
#création de tranche d'age
df_customers["age_group"] = pd.cut(df_customers["age"],bins=8, labels=["18-28", "29-38", "39-48", "49-58", "59-68", "69-78", "89-98", "99-108"])

<div style="display: flex; background-color: #a75d38;" >
<h1 style="margin: auto; padding: 20px; ">3.2 - Fichiers produits</h1>
</div>

In [12]:
#vérification que les id_produits sont aux bons format et qu'ils sont tous uniques
df_products_test = df_products.copy()
df_products_test["start_id_prod"] = df_products_test["id_prod"].str[0:2]
print(f"Les différents début d'identifiant produit existant sont : {df_products_test['start_id_prod'].unique()}")
print(f"Il y a {len(df_products['id_prod'].unique())} identifiants uniques, soit autant que de produits différents.")

Les différents début d'identifiant produit existant sont : ['0_' '1_' '2_' 'T_']
Il y a 3287 identifiants uniques, soit autant que de produits différents.


In [34]:
#vérification de la colonne catégorie
print(df_products["categ"].unique())

[0 1 2]


In [13]:
#vérification de la cohérence des prix 
print(f"Il y a {len(df_products.loc[df_products['price'] < 0])} prix inférieur à 0.")
std_price = df_products["price"].std()
avg_price = df_products["price"].mean()
print(f"La moyenne de prix est {round(avg_price,2)} et l'ecart-type {round(std_price,2)}")
print(f"Il y a {len(df_products.loc[(df_products['price'] > avg_price + 3*std_price) | (df_products['price'] < avg_price - 3*std_price)])} produits qui ont un Z-score élevé")
# Après vérification, le Z-score maximum pour un produit est d'environ 8.

Il y a 1 prix inférieur à 0.
La moyenne de prix est 21.86 et l'ecart-type 29.85
Il y a 97 produits qui ont un Z-score élevé


In [14]:
#correction de prix et d'identifiants 
print(df_products.loc[df_products['price'] < 0])
print(df_products.loc[df_products["id_prod"].str.startswith('T_')])
#Vérification de l'éventuel vente du produit concerné avant de le supprimer
print(df_transactions.loc[df_transactions["id_prod"].str.startswith('T_')])
#produits utilisé pour réalisé des tests, non pertinents pour l'analyse, nous pouvons le supprimer
df_products = df_products.loc[df_products["id_prod"] != "T_0"]

    id_prod  price  categ
731     T_0   -1.0      0
    id_prod  price  categ
731     T_0   -1.0      0
       id_prod                             date session_id client_id
3019       T_0  test_2021-03-01 02:30:02.237419        s_0      ct_0
5138       T_0  test_2021-03-01 02:30:02.237425        s_0      ct_0
9668       T_0  test_2021-03-01 02:30:02.237437        s_0      ct_1
10728      T_0  test_2021-03-01 02:30:02.237436        s_0      ct_0
15292      T_0  test_2021-03-01 02:30:02.237430        s_0      ct_0
...        ...                              ...        ...       ...
657830     T_0  test_2021-03-01 02:30:02.237417        s_0      ct_0
662081     T_0  test_2021-03-01 02:30:02.237427        s_0      ct_1
670680     T_0  test_2021-03-01 02:30:02.237449        s_0      ct_1
671647     T_0  test_2021-03-01 02:30:02.237424        s_0      ct_1
679180     T_0  test_2021-03-01 02:30:02.237425        s_0      ct_1

[200 rows x 4 columns]


<div style="display: flex; background-color: #a75d38;" >
<h1 style="margin: auto; padding: 20px; ">3.3 - Fichiers transactions</h1>
</div>

In [15]:
#suppression des transactions liées aux identifiants "ct_0" et "ct_1" pour pouvoir vérifier le parsage des dates
df_transactions = df_transactions.loc[~df_transactions["client_id"].isin(["ct_0", "ct_1"])]

In [16]:
#verifie que les dates sont correctement parser
pd.to_datetime(df_transactions["date"], format="%Y/%m/%d", errors="raise")

0        2022-05-20 13:21:29.043970
1        2022-02-02 07:55:19.149409
2        2022-06-18 15:44:33.155329
3        2021-06-24 04:19:29.835891
4        2023-01-11 08:22:08.194479
                    ...            
679527   2022-01-15 13:05:06.246925
679528   2022-03-19 16:03:23.429229
679529   2022-12-20 04:33:37.584749
679530   2021-07-16 20:36:35.350579
679531   2022-09-28 01:12:01.973763
Name: date, Length: 679332, dtype: datetime64[ns]

In [17]:
#supprime les lignes sans valeurs
df_transactions = df_transactions.dropna(how="all", axis=0)

In [18]:
#vérification de la colonne client_id
list_clients = df_customers["client_id"].unique().tolist()
print(f"Il y a {len(df_transactions.loc[df_transactions['client_id'].isin(list_clients) == False])} transactions qui ont un client_id erronées")

Il y a 0 transactions qui ont un client_id erronées


In [19]:
#vérification de la colonne id_prod
list_produits = df_products["id_prod"].unique().tolist()
print(f"Il y a {len(df_transactions.loc[df_transactions['id_prod'].isin(list_produits) == False])} transactions qui ont un identifiant produit erronées")

Il y a 221 transactions qui ont un identifiant produit erronées


In [20]:
# Un seul id_prod concerné par les transactions n'étant relié à aucun id_pro
df_transactions.loc[df_transactions['id_prod'].isin(list_produits) == False,"id_prod"].unique()

array(['0_2245'], dtype=object)

<div style="display: flex; background-color: #7a310c;" >
<h1 style="margin: auto; padding: 20px; ">4. Jointures et analyse préliminaires</h1>
</div>

<div style="display: flex; background-color: #a75d38;" >
<h1 style="margin: auto; padding: 20px; ">4.1 - Jointures des fichiers</h1>
</div>

In [32]:
#jointure à gauche du fichier de transactions avec les produits 
df = pd.merge(df_transactions, df_products, on = "id_prod", how="left", indicator=True)
df.rename(columns={"_merge" :"_merge_1"}, inplace=True)
#jointure à gauche du fichier merger et des clients
df = pd.merge(df, df_customers, on = "client_id", how="left", indicator=True)

In [33]:
df.head()

,id_prod,date,session_id,client_id,price,categ,_merge_1,sex,age,age_group,_merge
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0,both,f,39,29-38,both
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,15.99,1.0,both,m,37,29-38,both
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,7.99,0.0,both,f,57,49-58,both
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,69.99,2.0,both,m,25,18-28,both
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,4.99,0.0,both,m,45,39-48,both


In [35]:
#vérification de la jointure
print(len(df.loc[df["_merge"] != "both"]))
print(len(df.loc[df["_merge_1"] != "both"]))
# les 221 transactions qui posent problèmes viennent toutes du produits "0_2245"

0
221


In [44]:
#analyse des catégories de produits
#catégorie 0
print("Moyenne catéorie 0 : ",df.loc[df["categ"] == 0, "price"].mean())
print("Minimum et maximum catégorie 0 : min = ", df.loc[df["categ"] == 0, "price"].min(), "max = ",df.loc[df["categ"] == 0, "price"].max())
# catégorie 1
print("Moyenne catéorie 1 : ",df.loc[df["categ"] == 1, "price"].mean())
print("Minimum et maximum catégorie 1 : min = ", df.loc[df["categ"] == 1, "price"].min(), "max = ",df.loc[df["categ"] == 1, "price"].max())
# catégorie 2
print("Moyenne catéorie 2 : ",df.loc[df["categ"] == 2, "price"].mean())
print("Minimum et maximum catégorie 2 : min = ", df.loc[df["categ"] == 2, "price"].min(), "max = ",df.loc[df["categ"] == 2, "price"].max())

Moyenne catéorie 0 :  10.638188052250644
Minimum et maximum catégorie 0 : min =  0.62 max =  40.99
Moyenne catéorie 1 :  20.485729522954276
Minimum et maximum catégorie 1 : min =  2.0 max =  80.99
Moyenne catéorie 2 :  76.20741221938987
Minimum et maximum catégorie 2 : min =  30.99 max =  300.0
